In [1]:
# 常用的引用和创建对象实体
from pyspark import SparkConf,SparkContext
conf = SparkConf().setMaster("local").setAppName("My App")
sc = SparkContext(conf = conf)

# 1, 简介
1. 累加器（accumulator）: 对信息进行聚合
2. 广播变量（broadcast variable）: 高效分发较大的对象
3. 批操作
4. 外部程序的交互方式


- **共享变量是一种可以在Spark任务中使用的特殊类型的变量**
- **当任务需要很长的时间的时候，在你多个数据元素之间共享一次配置就会比较有效率**
- Spark可以使用**pipe()**方法来与其他程序通过标准输入和标准输出进行交互


# 2, 类加器 Accumulator
- 提供了将工作节点中的值聚合到驱动器程序中的简单语法，即一种**共享变量**
- 调用方式如下：
   1. 在Driver中调用**SparkContext.accumulator(initialValue)的方法，创建出存有初始值的累加器,返回值为org.apache.spark.Accumulator[T]的对象，T是初始类型
      - `blankLines = sc.accumulator(0)`
      - 创建初始值为0的累加器*blankLines*
      - 在某些程序中调用累加器之前需要 *glibal* 全局声明所有的累加器变量
   2. 在Spark闭包程序中执行 +=1 对累加器进行自增值
      - `blankLines += 1`
   3. Driver可以调用累加器的**value**属性来访问累加器的值
- **工作节点上的任务不能访问累加器的值，累加器是一个只写的变量，此模式下累加器的实现可以更加高效，不需要对每次更新操作进行复杂的通信**

## 2.1 累加器与容错性
- Spark会自动重新执行失败或者比较慢的任务来应对有错误的或者比较慢的机器
- 即使没有节点失败，Spark有时也需要重新运行任务来获取缓存中被移除内存的数据
- **对于要在Actions操作中使用的累加器，Spark只会把每个任务对各累加器的修改应用一次**
- 如果想要一个**无论在失败还是在重复计算时都可靠的累加器**,我们必须把它放在**foreach()**这样的**Action**操作中
- 对于在**RDD Transformation**操作中使用的累加器，就不能保证有这种情况了
- **Transformation**操作中使用的累加器可能会发生不止一次的更新，在**Transformation**操作中，累加器常常用于调试目的。

## 2.2 自定义累加器
- 可以定义任何累加器的类型，和累加器的操作，如选择最大的累加器值等等

# 3, 广播变量 broadcast vairable
- 也是一种共享变量
- **让程序高效地向所有工作节点发送一个较大的*[只读值]*，以供一个或多个Spark操作使用**
- Spark会自动把**闭包中所有引用到的变量**都放发送到工作节点上，虽方便，但低效：
   1. 默认的任务发射机制是专门为小任务进行优化的
   2. 事实上，程序可能会在多个并行操作中使用同一个变量，但是Spark回为每个操作分别分发。
- **广播变量**
   1. 类型为 **spark.broadcast.Broadcast[T]** 的一个对象
   2. 存放着类型为 T 的值
   3. 可以在任务中通过对**Broadcast**对象调用**value**来获取该对象的值
   4. 这个值只会被发送到各个节点一次，使用的是一种高效的类似于BitTorrent的通信机制
- **广播变量的使用过程**
   1. 通过一个类型 T 的对象调用**SparkContexrt.broadcast**创建出一个**Broadcast[T]**对象，任何和序列化的类型都可以实现
      - `signPrefixes = sc.broadcast(loadCallSignTable())`
   2. 通过**value**属性访问该对象的值
   3. 变量只会被发送到各个节点一次，应作为只读值处理（修改这个值不会影响到别的节点）
   4，只能在**Driver**中进行修改
- 广播的优化
   1. 当广播一个较大的值时，选择既快又好的序列化格式是很重要的，因为如果序列化对象的时间很长，或者传送花费的时间太久，这段时间就会成为性能瓶颈
   2. 最好使用基本类型的数组，也可以使用**spark.serializer**属性选择另一个序列化库来优化序列化的过程
   3. 也可以自己实现序列化过程：Python的pickle库自定义序列化

# 4, 基于分区进行操作
- 基于分区的操作可以让我们避免为每一个数据元素进行重复的配置工作。
- Spark提供基于分区的**map**和**foreach**操作，让你的部分代码只对RDD的每个分区运行一次，这样可以帮助降低这些操作的代价
- 基于分区操作RDD时，Spark会为函数提供该分区的元素的迭代器
- 返回值方面，也返回一个迭代器。
- 按分区执行的操作符：
    - <img src="https://raw.githubusercontent.com/ColinsGitCode/JupyterNotebook/4976c04732ac8018a51dc0ea818ac7145c86e92e/ipynbFiles/Materials/%E6%8C%89%E5%88%86%E5%8C%BA%E6%89%A7%E8%A1%8C%E7%9A%84%E6%93%8D%E4%BD%9C%E7%AC%A6.jpg" />
- 


# 5, 与外部程序间的管道
- RDD.pipe()方法。从外部程序的方法或者库来对RDD中的每个元素进行一些特定的处理


# 6, 数值RDD的操作
- Spark对包含数值数据的RDD提供了一些描述性的统计操作。
- Spark的数值操作都是通过流式计算实现的，允许以每次一个元素的方式构建出模型
- 这些统计数据都会在调用**stats()**时通过一次遍历数据计算出来并以**StatsCounter**对象返回
- **StatsCounter中可用的汇总统计数据**
   1. count()           RDD中的元素的个数
   2. mean()            元素的平均值
   3. sum()             综合
   4. max()             最大值
   5. min()             最小值
   6. variance()        元素的方差
   7. sampleVariance()  从采样中计算出的方差
   8. stdev()           标准差
   9. sampleStdev()     采样的标准差
-  如果只想计算其中的一个，则直接调用即可 RDD.sun()